In [ ]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import (
    BaseMessage,
    SystemMessage,
    HumanMessage,
    AIMessage,
)

# SystemMessage -> BaseMessage -> Serializable
# HumanMessagePromptTemplate -> BaseMessagePromptTemplate -> Serializable

### Prompt for Completion Model

In [21]:
template = """Describe {plant}.
First, think whether {plant} exist.
If they {plant} don't exist, answer "I don't have enough information
about {plant}".
Otherwise, give their title, a short summary and then talk about
origin and cultivation.
After that, describe their physical characteristics.
"""

In [ ]:
# PromptTemplate -> StringPromptTemplate -> BasePromptTemplate -> RunnableSerializable
# with required "plant" variable
prompt_template = PromptTemplate(
    input_variables=["plant"],
    template=template,
)

In [22]:
# populate "plant" variable
prompt_template = PromptTemplate(
    partial_variables={"plant":"black cucumbers"},
    template=template,
)

In [23]:
prompt_template = PromptTemplate.from_template(template)

In [ ]:
prompt_template.pretty_print()

In [ ]:
prompt = prompt_template.format(plant="black cucumbers")
print(prompt)

In [ ]:
prompt_value = prompt_template.format_prompt(plant="black cucumbers")
for message in prompt_value.to_messages():
    print(message.type, ":", message.content)

In [ ]:
prompt_value = prompt_template.invoke({"plant": "black cucumbers"})
print(prompt_value.to_string())

### Prompt for Chat Model

In [ ]:
# messages: Sequence[MessageLikeRepresentation]
# MessageLike = Union[BaseMessagePromptTemplate, BaseMessage, BaseChatPromptTemplate]
# MessageLikeRepresentation = Union[
#     MessageLike,
#     tuple[
#         Union[str, type],
#         Union[str, list[dict], list[object]],
#     ],
#     str,
#     dict,
# ]
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                """You are a helpful assistant that helps extract entities from 
product descriptions. You always respond in a json format.""")
        ),
        HumanMessagePromptTemplate.from_template(
            """Extract the following entities:\n{entities}\n 
from the item's description:\n{description}."""),
    ]
)

In [ ]:
messages = chat_template.format_messages(entities="price, RAM", description="Price: $1,000 and RAM: 32GB")
for message in messages:
    print(message.type, message.content)

In [ ]:
prompt_value = chat_template.format_prompt(entities="price, RAM", description="Price: $1,000 and RAM: 32GB")
messages = prompt_value.to_messages()
print(prompt_value.to_string())

In [ ]:
prompt_value =  chat_template.invoke({"entities":"price, RAM", "description":"Price: $1,000 and RAM: 32GB"})
print(prompt_value.to_string())

In [40]:
template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke(
    {
        "name": "Bob",
        "user_input": "What is your name?"
    }
)

messages = prompt_value.to_messages()
for message in messages:
    print(message.type, ":", message.content)

system : You are a helpful AI bot. Your name is Bob.
human : Hello, how are you doing?
ai : I'm doing well, thanks!
human : What is your name?


In [ ]:
template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot."),
    # Means the template will receive an optional list of messages under
    # the "conversation" key
    ("placeholder", "{conversation}")
    # Equivalently:
    # MessagesPlaceholder(variable_name="conversation", optional=True)
])

prompt_value = template.invoke({})

messages = prompt_value.to_messages()
for message in messages:
    print(message.type, ":", message.content)

In [ ]:
prompt_value = template.invoke(
    {
        "conversation": [
            ("human", "Hi!"),
            ("ai", "How can I assist you today?"),
            ("human", "Can you make me an ice cream sundae?"),
            ("ai", "No.")
        ]
    }
)

messages = prompt_value.to_messages()
for message in messages:
    print(message.type, ":", message.content)

### Few-Shot Prompt

In [ ]:

examples = [
    {
        "keywords": """best fan for hot summer days, powerful, cozy,
wood tone, enjoy cold drink""",
        "response": (
            "Slogan: Breeze 4000: Feel the Difference.\n"
            "Tagline: Design, Comfort, Performance – The Ultimate Summer Upgrade.\n"
            "Short Description: Beat the heat in style with the Breeze 4000. "
            "Its sleek wood-tone design and whisper-quiet operation create "
            "the perfect oasis for enjoying a cool drink on those hot summer days.\n"
            "Call to action: 1/ Experience the Breeze 4000 difference today. "
            "(Emphasizes the unique qualities)\n"
            "2/ Upgrade your summer. Shop the Breeze 4000 now. (Creates a sense of urgency)\n"
            "3/ Find your perfect Breeze 4000 style. (Focus on design and personalization)"
        )
    },
]

example_prompt = PromptTemplate(
    input_variables=["keywords", "response"],
    template="Example keywords:\n{keywords}\nExample response:\n{response}"
)


In [ ]:
instruction = (
    """---INSTRUCTION--- \nYou are an intelligent assistant that helps
marketers write great copy for campaigns on our website, which sells
premium ceiling fans to design-conscious customers. Please create
campaign copy (a slogan, a tagline, a short description, and three
calls-to-action) based on keywords. Use the information from your
context to choose the right products to advertise. Follow the examples
below to ensure that you follow company branding standards.\n"""
)

prompt_template = "---CONTEXT---\n{context}\n---KEYWORDS FOR CREATING COPY---\n{keywords}\n"

prompt = FewShotPromptTemplate(
    prefix=instruction,
    
    # iterate through examples
    examples=examples,
    example_prompt=example_prompt,

    suffix=prompt_template,
    input_variables=["context", "keywords"],
)

In [ ]:

context = [
    {
        "name": "Whirlwind BreezeMaster 3000",
        "performanceRating": "high",
        "outdoor": True,
        "powerSource": "electric",
        "price": 249.99
    }
]

keywords = "best fan for dry heat, powerful, outdoor, porch, affordable"

prompt_value = prompt.invoke({"context": context, "keywords": keywords})

messages = prompt_value.to_messages()
print("Message count", ":", len(messages), "\n")
for message in messages:
    print(message.type, ":", message.content)